In [1]:
import pickle
import pandas as pd
import numpy as np

def load_data(dataset='training'):    
    return pd.read_pickle('../data_processed/' + dataset + '_set.pkl')

def get_dimensions(mel_shape, shape='stacked'):
    if shape =='flat':   
        mel_depth = 1
        mel_height = 256
    elif shape == 'stacked':
        mel_depth = 2
        mel_height = 128        
    
    mel_width = int(mel_shape[1])    
    return mel_height, mel_width, mel_depth

In [2]:
def process_files(dataset='training', features=['Mel'], shape='mel_only', window_size=28):
    
    df = load_data(dataset=dataset)
        
    #Where it will be stored
    files, labels, data = [],[],[]
    
    #List of file names in the dataset
    file_names = list(df.File_id.unique())
    
    for index, row in df.iterrows():
        
        #Load the needed columns, and stack them, move the volume dim to the end
        mel = np.array(row[features])
        mel = np.stack((mel)) 
        
        #obtain some dimentions about the set to load
        if len(features) > 1:
            mel_height, mel_width, mel_depth = get_dimensions(shape=shape, mel_shape=mel.shape)
        else:
            mel_height, mel_width, mel_depth = mel.shape[1], mel.shape[2], mel.shape[0]
        
        #each mel needs to be chopped into segments of window_size width
        batch_size = int(mel.shape[2] / window_size)
        
        #reshape mel and remove parts that will be ignored
        mel = np.reshape(mel[:,:,0:batch_size*window_size], (mel_depth, mel_height, batch_size*window_size))                
        
        for i in list(range(batch_size)):
            labels.append(row['Label'])
            files.append(row['File_id'])           
            data.append(mel[:,:,i*window_size:(i+1)*window_size])                        
            
    return np.array(data, dtype=np.float32), np.array(labels), np.array(files)

In [4]:
x_train, y_train, f_train = process_files(dataset='training', features=['Mel', 'Mel_deltas'], 
                                                    shape='stacked', window_size=28)
x_test, y_test, f_test = process_files(dataset='test', features=['Mel', 'Mel_deltas'], 
                                                    shape='stacked', window_size=28)

In [5]:
x_test.shape

(577, 2, 128, 28)